In [ ]:
!pip install pymongo
import requests
import re
import json
from bson import json_util
from google.colab import files
import io

In [ ]:
# Function to convert Mongo Extended JSON to regular JSON
def mongoextjson_to_json(raw_data):
    # Convert MongoDB's ObjectId to standard JSON-compliant format
    json_data = re.sub(r'ObjectId\s*\(\s*\"(\S+)\"\s*\)', r'{"$oid": "\1"}', raw_data)


    # Add this block:
    try:
        data = json.loads(json_data, object_hook=json_util.object_hook)
        return data  # Return if successful
    except json.JSONDecodeError:
        pass  # If parsing as a single object fails, proceed

    data = []
    for line in json_data.splitlines():
        if line.strip():
            try:
                data.append(json.loads(line, object_hook=json_util.object_hook))
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {line}, Error: {e}")
    return data

In [ ]:
# Function to read and convert MongoDB Extended JSON from file content
def process_local_json(file_content):
    raw_data = file_content.read()
    return mongoextjson_to_json(raw_data)

In [ ]:
# Upload brands.json
uploaded_brands = files.upload()
brands_file_content = io.StringIO(uploaded_brands[list(uploaded_brands.keys())[0]].decode('utf-8'))


Saving brands.json to brands (1).json
Saving users.json to users (1).json


In [ ]:
# Upload users.json (assuming the file is named 'users.json')
uploaded_users = files.upload()
users_file_content = io.StringIO(uploaded_users[list(uploaded_users.keys())[0]].decode('utf-8'))

# Now you can process the users' data:
users_data = process_local_json(users_file_content)

Saving brands.json to brands (2).json
Saving users.json to users (2).json


In [ ]:
# Process Users and Brands files
brands_data = process_local_json(brands_file_content)
users_data = process_local_json(users_file_content)

In [ ]:
# Upload and process receipts.json
uploaded_receipts = files.upload()
receipts_file_content = io.StringIO(uploaded_receipts[list(uploaded_receipts.keys())[0]].decode('utf-8'))
receipts_data = process_local_json(receipts_file_content)

Saving receipts.json to receipts.json


In [ ]:
import pandas as pd


# Create DataFrames
receipts_df = pd.DataFrame(receipts_data)

# Extract items data
items_data = receipts_df['rewardsReceiptItemList'].apply(pd.Series).stack().reset_index(level=1, drop=True)
items_df = pd.DataFrame(items_data.to_list(), columns=items_data.iloc[0].keys())

# Create receipts_updated (without nested items)
receipts_updated_df = receipts_df.drop(columns=['rewardsReceiptItemList'])

# Now you have receipts_updated_df and items_df
print(receipts_updated_df.head())
print(items_df.head())

In [ ]:
# Create DataFrames
brands_df = pd.DataFrame(brands_data)
users_df = pd.DataFrame(users_data)

In [ ]:
# Export receipts_updated_df to CSV
receipts_updated_df.to_csv('receipts_updated.csv', index=False)

# Export items_df to CSV
items_df.to_csv('items.csv', index=False)

In [ ]:
# Create DataFrames
brands_df = pd.DataFrame(brands_data)
users_df = pd.DataFrame(users_data)
receipts_df = pd.DataFrame(receipts_data)

# Extract items data
# items_data = receipts_df['rewardsReceiptItemList'].apply(pd.Series).stack().reset_index(level=1, drop=True)
# items_df = pd.DataFrame(items_data.to_list(), columns=items_data.iloc[0].keys())
# Extract items data with receiptId
# Extract items data with receiptId at the front

items_data = receipts_df['rewardsReceiptItemList'].apply(pd.Series).stack().reset_index(level=1, drop=True)
items_df = pd.DataFrame(items_data.to_list(), columns=items_data.iloc[0].keys())

# Add receiptId column to items_df, handling variations in _id column
items_df['receiptId'] = receipts_df['_id'].apply(lambda x: str(x.get('$oid', x)) if isinstance(x, dict) else str(x)).repeat(receipts_df['rewardsReceiptItemList'].apply(lambda x: len(x) if isinstance(x, (list, dict)) else 0)).reset_index(drop=True)

# Reorder columns to place receiptId at the front
desired_columns = ['receiptId'] + list(items_df.columns[:-1])  # Assuming receiptId is added as the last column
items_df = items_df[desired_columns]


# Create receipts_updated (without nested items)
receipts_updated_df = receipts_df.drop(columns=['rewardsReceiptItemList'])

# Export to CSV
brands_df.to_csv('brands.csv', index=False)
users_df.to_csv('users.csv', index=False)
receipts_updated_df.to_csv('receipts_updated.csv', index=False)
items_df.to_csv('items.csv', index=False)

# Download all CSV files (optional)
from google.colab import files

files.download('brands.csv')
files.download('users.csv')
files.download('receipts_updated.csv')
files.download('items.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>